In [61]:
import mysocket as sock
import requests
import re

def printByLine(s):
    pattern = r'\r\n'
    lines = re.split(pattern, s)
    for line in lines[:-1]:
        real_line = line + '\r\n'
        print(repr(real_line))


protocol = "http"
host = "hadoop.mathsci.denison.edu"
port = 80
buildURL = lambda s: "{}://{}:{}/{}".format(protocol, host, port, s)

In [62]:
requestlines = ["GET /bookweb/data/basic.html HTTP/1.0",
                "Host: hadoop.mathsci.denison.edu",
                "Connection: close",
                ""
               ]
request = "\r\n".join(requestlines) + "\r\n"
request

'GET /bookweb/data/basic.html HTTP/1.0\r\nHost: hadoop.mathsci.denison.edu\r\nConnection: close\r\n\r\n'

In [63]:
printByLine(request)

'GET /bookweb/data/basic.html HTTP/1.0\r\n'
'Host: hadoop.mathsci.denison.edu\r\n'
'Connection: close\r\n'
'\r\n'


In [67]:
webconn = sock.makeConnection("hadoop.mathsci.denison.edu", 80)
sock.sendString(webconn, request)
s = sock.receiveTillClose(webconn)

In [64]:
webconn = sock.makeConnection("hadoop.mathsci.denison.edu", 80)
for line in requestlines:
    sock.sendString(webconn, line)
    sock.sendCRLF(webconn)
s = sock.receiveTillClose(webconn)

In [68]:
s

'HTTP/1.1 200 OK\r\nServer: nginx/1.14.0 (Ubuntu)\r\nDate: Fri, 03 Apr 2020 13:22:56 GMT\r\nContent-Type: text/html\r\nContent-Length: 162\r\nLast-Modified: Fri, 03 Apr 2020 12:38:55 GMT\r\nConnection: close\r\nETag: "5e872e5f-a2"\r\nExpires: Fri, 03 Apr 2020 14:22:56 GMT\r\nCache-Control: max-age=3600\r\nAccept-Ranges: bytes\r\n\r\n<!DOCTYPE html>\n<html>\n<head>\n<title>Denison CS-181/DA-210</title>\n</head>\n<body>\n\n<h1>Welcome</h1>\n\n<p>Great to have you taking this class!</p>\n\n</body>\n</html>\n'

In [53]:
url = buildURL("bookweb/data/basic.html")
url

'http://hadoop.mathsci.denison.edu:80/bookweb/data/basic.html'

In [54]:
response = requests.get(url)

In [55]:
response.status_code

200

## Student Practice

Do each of the following first from web browser, then from requests module.

- simple http get request of html from root of a location
- http get request of html from non-root (but default)
- http get request of html from non-root non-default
- request for non-existant resource
- http get request of csv

More advanced HTTP operations
- http get request with parameters
- http post request with parameters
- request for redirected resource
- request for protected resource

## Programmatic HTTP `GET` request

[`requests` module documentation](https://requests.readthedocs.io/en/master/)

In [1]:
import requests

Based on  our newfound understanding of a basic URL consisting of parts of a protocol, a location, and a resource identifier, we use separate variables for each and compose them using a format method on a template string:

### Basic request of URL from root of a location

In [2]:
protocol = 'http://'
location = 'personal.denison.edu'
resource = '/'

template = '{}{}{}'
url = template.format(protocol, location, resource)

url

'http://personal.denison.edu/'

The simplest use of the `requests` module is perform a `GET` method using the `get()` function of the module, with a simple single argument of a string specifying the URL.

In [3]:
resp = requests.get(url)

**Whenever** we make a request, we **must** check for a successful return by looking at the `status_code` attribute of the response object.  If we get a non-successful code, our programming must either fix a bug, or recover from the error, informing the user of the problem.

In [4]:
if resp.status_code != 200:
    print('Error retieving request from', url)

In many cases, before we act on the **message body**, which is the **content** on what has been retrieved from the server, we want or need to know about additional metadata, like:
- How large is the content
- What type of data has been returned (html, csv, xml, xhtml, json, etc.)
- other information such as date, server, etc.

The `headers` attribute of the response object contains a dictionary containing all the meta-data information provided by the server that came from the header field part of the HTTP response.  Note that HTTP does not know about Python dictionaries ... the transformation/translation from the HTTP response, which is just a set of characters in the underlying data stream connection between the client and the server, happens because of the `requests` module. 

In [ ]:
resp.headers

Finally, we can access the data of the response in two ways.  If it is textual data, and we want the string version of the data (appropriate to the encoding), we use the `text` attribute of the response object.

In [ ]:
resp.text

While if the data is binary (like an image, or a compiled program), or if we want the form without applying the encoding, we use the `content` attribute of the response object.

In [ ]:
resp.content

Notice the `b` prefix in the demonstration above, indicating that these are raw bytes.

A `requests` module response object also has an attribute (`request`) that references the original request corresponding to the this particular response.  Some of the request's interesting attributes include:
- `headers`
- `path-url`
- `url`
- `method`
- `body`

In [ ]:
resp.request.headers

In [ ]:
resp.request.path_url

In [ ]:
resp.request.url

In [ ]:
resp.request.method

In [ ]:
resp.request.body

### Basic URL request to non-root of a location

We can change our resource to request the HTML using a resource identifier other than the root:

In [ ]:
resource = '/~kretchmar/'

template = '{}{}{}'
url = template.format(protocol, location, resource)

url

**A1** Add code to retrieve from this new url and then print the **length** of the content returned and print the first 100 characters in the body of the returned HTML.

**A2** Using Chrome, copy the HTTP raw request (the GET, including all header lines), and then copy the response, from the response line through the response headers.

### URL request to a non-root resource identifier that does not exist

**B1** Now change the resource to something that does not exist, and demonstrate how your code can detect when something does not exist and print an appropriate error message to the user.

**B2** Using chrome, copy and paste the HTTP raw request and also the HTTP raw response corresponding to exactly the same request made above.

### URL request to a non-HTML, non-default resource identified location

The goal here is to find a resource, say within `discovercs.denison.edu` that gives us a URL for an explicit file provided by a Data System provider.  So look for a csv or text file and determine the URL.  Then give two cells below, one which requests the resource programmatically and uses the headers and content to print information about the response (size, type) and some of its data body.  Don't forget to include code to check the status return as well.

The second cell should again use Chrome to show and help understand the HTTP request and response going on.

### URL request to a JSON that uses URL parameters

Resources that we request of a Data System provider may not always be for an existing file (text or otherwise).  Just like when we make requests to an SQL server and specify (through our SQL SELECT query) some part of the data we are interested in, and the server processes from the stored files and tables and returns just the requested subset, we need such dynamic flexibility in the Data System providers that use HTTP.

This means that we need to pass additional information through HTTP, but still adhere to the methods/verbs that the protocol provides.  One means of passing additional information is to extend the use of our URL to include one or more key-value pairs.  These can be used by the server/provider, which can dynamically create our response.